# Load Collectives and Load Histograms

From the load (stress) side pyLife provides the classes [`LoadCollective`](https://pylife.readthedocs.io/en/stable/stress/load_collective.html) and [`LoadHistogram`](https://pylife.readthedocs.io/en/stable/stress/load_histogram.html) to deal with load collectives.  `LoadCollective` contains individal hysteresis loops whereas `LoadHistogram` contains a 2D-histogram of classes of hysteresis loops and the number of cycles with which they occur.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import pylife.stress.timesignal as TS
import pylife.stress.rainflow as RF
import pylife.strength.meanstress as MS
import pylife.strength.fatigue
plt.rcParams['figure.figsize'] = [12, 7.5]

## A simple load signal

Let's take a look at a really simple load signal:

In [ ]:
load_signal = np.array([0., 2.0, -2.0, 1.0, -1.0, 2.0, -2.0, 1.0, -1.0, 2.0, -2.0, 1.0, -1.0, 2.0, 0.])
plt.plot(load_signal)

Now let's perform a rainflow analysis.

In [ ]:
detector = RF.FourPointDetector(recorder=RF.LoopValueRecorder())
detector.process(load_signal)

The detector now contains the recorder which recorded the hysteresis loops for us. The simple load collective comes as a attribute of the detector:

In [ ]:
collective = detector.recorder.collective
collective

As you can see, the rainflow analysis found five hystresis loops, three from 1.0 to -1.0 and two from -2.0 to 2.0. Alternatively you can ask the recorder for a load histogram:

In [ ]:
histogram = detector.recorder.histogram(bins=6)
histogram

This is a bit hard to read. What you see is a `pands.Series` that has a two dimensional `IntervalIndex` as index. The histogram is all empty except the two classes `from`: (-2.0, 1.5] `to`: (1.5, 2.0] has 2.0 cycles and `from`: (0.5, 1.0] `to`: (-1.0, -1.5] has 3.0 cycles. Tose correspond to the two loops from -2.0 to 2.0 and the three loops 1.0 to -1.0.

## Working with load collectives and load histograms

A load collective and a load histogram can be processed by the two classes `LoadCollective` and `LoadHistogram`. Both inherit from the common base class `AbstractLoadCollective`. There is the common accessor attribute `load_collective` that convert a pandas object with the load collective resp. load histogram data into the corresponding class.

First let's look at a load collective. You can easily calculate the amplitude of each hysteresis loop:

In [ ]:
cl = collective.load_collective
cl.amplitude

Same for the mean stress and the R-value:

In [ ]:
cl.meanstress, cl.R

There is also the attribute `cycles`:

In [ ]:
cl.cycles

As you can see, the cycles are all 1.0 because we have an entry for each indivudual hysteresis loop which by definition occurs only once.

Now let's take a look at the histogram:

In [ ]:
hi = histogram.load_collective
hi.amplitude, hi.meanstress, hi.R

This might look a bit confusing as this only shows the amplitudes, meanstresses and R-values correspond to the bins of the histogram. Remember, that they were all except two empty. So let's restrict the histogram to bins that are not empty:

In [ ]:
not_empty = histogram > 0.0
hi.amplitude[not_empty], hi.cycles[not_empty]

The amplitude values 1.75 and 0.75 correspond to 2.0 and 1.0. They are in the middle of the histogram bins.

## A more complex example

Now let's take a look at a more complex load collective. We use the `TimeSignalGenerator` to generate a load signal.

In [ ]:
load_signal = TS.TimeSignalGenerator(
    10, 
    {
        'number': 50,
        'amplitude_median': 1.0, 'amplitude_std_dev': 0.5,
        'frequency_median': 4, 'frequency_std_dev': 3,
        'offset_median': 0, 'offset_std_dev': 0.4
    }, None, None
).query(50000)
plt.plot(load_signal)

Again we perform a rainflow analysis to obtain the load histogram.

In [ ]:
detector = RF.FourPointDetector(recorder=RF.LoopValueRecorder())
detector.process(load_signal)

histogram = detector.recorder.histogram(64)

We can plot the histogram with a bit of processing.

In [ ]:
fr, to = histogram.index.levels[0], histogram.index.levels[1]
numpy_hist = np.flipud(histogram.values.reshape(len(fr),len(to)))
X, Y = np.meshgrid(fr.left, to.left)
plt.pcolormesh(X, Y, numpy_hist)

In order to use this load histogram for a damage calculation we can perform a mean stress transformation to transorm all the hysteresis loops to one given R-value.

In [ ]:
meanstress_sensitivity = pd.Series({
    'M': 0.3,
    'M2': 0.2
})

transformed_histogram = histogram.meanstress_transform.fkm_goodman(meanstress_sensitivity, R_goal=-1)
transformed_histogram.to_pandas()

We can also plot the cumulated version of the histogram. Therefore we put the amplitude and the cycles into a dataframe.

In [ ]:
df = pd.DataFrame({
    'cycles': transformed_histogram.cycles, 
    'amplitude': transformed_histogram.amplitude, 
}).sort_values('amplitude', ascending=False)

Now we can plot the amplitude against the cumulated sum of the cycles:

In [ ]:
plt.plot(np.cumsum(df.cycles), df.amplitude)
plt.loglog()